## computer vision

In [9]:
import cv2

def compute_similarity_histogram(img1_path, img2_path):
    # Load images
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    if img1 is None or img2 is None:
        raise ValueError("One or both image paths are invalid.")
    
    # Convert images to HSV colour space
    hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    
    # Compute 2D histograms for H and S channels
    hist1 = cv2.calcHist([hsv1], [0, 1], None, [50, 60], [0, 180, 0, 256])
    hist2 = cv2.calcHist([hsv2], [0, 1], None, [50, 60], [0, 180, 0, 256])
    
    # Normalise histograms
    cv2.normalize(hist1, hist1, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
    
    # Compare histograms using correlation
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return similarity

if __name__ == "__main__":
    image1_path = "image1.jpg"
    image2_path = "image2.jpg"
    
    sim_score = compute_similarity_histogram(image1_path, image2_path)
    print(f"Similarity Score: {sim_score:.2f}")

Similarity Score: 0.87


## First pass through

In [10]:
import torch
import torch.nn as nn
import cv2
import numpy as np

###############################################################################
# 1) LOAD YOLOv5n MODEL
###############################################################################
def load_model():
    """
    Loads the YOLOv5n model from Torch Hub and sets it to evaluation mode.
    """
    model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)
    model.eval()
    return model

###############################################################################
# 2) RECURSIVELY FLATTEN ALL SUBMODULES
###############################################################################
def flatten_submodules(module: nn.Module):
    """
    Recursively retrieve all submodules from a PyTorch module (BFS or DFS).
    Returns them as a list in the order they were visited.

    Example usage:
      all_layers = flatten_submodules(model)
      # then all_layers[0] is the first submodule, all_layers[1] is the second, etc.

    If you need to see each layer's name, you can adapt this to return (name, layer).
    """
    layers = []

    def recurse(m: nn.Module):
        # Add this module itself
        layers.append(m)
        # Recurse on children
        for child in m.children():
            recurse(child)

    recurse(module)
    return layers

###############################################################################
# 3) GET EMBEDDING BY HOOKING A SUBMODULE
###############################################################################
def get_embedding(model, image_path: str, layer_index: int):
    """
    Extracts a feature embedding from YOLOv5n by hooking into a specified
    submodule index from a flattened list of submodules.
    """
    # Load image via OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")

    # Convert BGR -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten all submodules of YOLO
    all_layers = flatten_submodules(model)

    if layer_index < 0 or layer_index >= len(all_layers):
        msg = (
            f"Invalid layer_index={layer_index}. Valid range: 0..{len(all_layers)-1}\n\n"
            "Hint: Inspect the layers or reduce your chosen index."
        )
        raise IndexError(msg)

    # Hook function to capture output from the chosen layer
    features = []
    def hook_fn(module, input, output):
        features.append(output)

    chosen_layer = all_layers[layer_index]
    hook = chosen_layer.register_forward_hook(hook_fn)

    # Run inference on a single image
    _ = model([img])

    # Remove hook
    hook.remove()

    if not features:
        raise RuntimeError("Feature extraction failed: The chosen layer produced no output.")

    # Flatten e.g. [batch, channels, H, W] -> [batch, channels*H*W]
    feat = features[0].view(features[0].size(0), -1)
    # Convert to NumPy (we only had one image in the batch)
    vec = feat.detach().cpu().numpy()[0]
    return vec

###############################################################################
# 4) COSINE SIMILARITY
###############################################################################
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray):
    """
    Compute cosine similarity between two vectors.
    Returns a float in [-1..1]. Closer to +1 => more similar.
    """
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return float(dot / (norm1 * norm2))

###############################################################################
# 5) MAIN DEMO
###############################################################################
def main():
    # Hardcoded paths to images
    image1_path = "image1.jpg"
    image2_path = "image2.jpg"
    image3_path = "image3.jpg"

    # CHOOSE A LAYER INDEX
    # Because we flatten the entire model, you can choose a wide range:
    # e.g. 0, 1, 2 ... up to len(all_layers)-1. The deeper layers tend
    # to produce more "semantic" features. Start with something like 50
    # or 100, depending on the total number of submodules.
    layer_index = 50

    print("Loading YOLOv5n model...")
    model = load_model()

    print("Extracting embeddings...")
    try:
        vec1 = get_embedding(model, image1_path, layer_index)
        vec2 = get_embedding(model, image2_path, layer_index)
        vec3 = get_embedding(model, image3_path, layer_index)
    except IndexError as e:
        print("\n[ERROR] Invalid layer index chosen.")
        print(e)
        # Optional: print submodule count or submodule names here
        # all_layers = flatten_submodules(model)
        # for i, layer in enumerate(all_layers):
        #     print(i, layer)
        return
    except Exception as e:
        print(f"[ERROR] {e}")
        return

    # Compute similarities
    sim_1_2 = cosine_similarity(vec1, vec2)
    sim_1_3 = cosine_similarity(vec1, vec3)

    print(f"\nCosine similarity between image1 and image2: {sim_1_2:.4f}")
    print(f"Cosine similarity between image1 and image3: {sim_1_3:.4f}")


if __name__ == "__main__":
    main()


Loading YOLOv5n model...


Using cache found in C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-1 Python-3.12.1 torch-2.5.0+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 
C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Extracting embeddings...

Cosine similarity between image1 and image2: 0.8229
Cosine similarity between image1 and image3: 0.7536


## Second pass through

In [11]:
import torch
import torch.nn as nn
import cv2
import numpy as np

###############################################################################
# 1) LOAD YOLOv5n MODEL
###############################################################################
def load_model():
    """
    Loads the YOLOv5n model from Torch Hub and sets it to evaluation mode.
    """
    model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)
    model.eval()
    return model

###############################################################################
# 2) RECURSIVELY FLATTEN ALL SUBMODULES
###############################################################################
def flatten_submodules(module: nn.Module):
    """
    Recursively retrieve all submodules from a PyTorch module (BFS or DFS).
    Returns them as a list in the order they were visited.

    Example usage:
      all_layers = flatten_submodules(model)
      # then all_layers[0] is the first submodule, all_layers[1] is the second, etc.

    If you need to see each layer's name, you can adapt this to return (name, layer).
    """
    layers = []

    def recurse(m: nn.Module):
        # Add this module itself
        layers.append(m)
        # Recurse on children
        for child in m.children():
            recurse(child)

    recurse(module)
    return layers

###############################################################################
# 3) GET EMBEDDING BY HOOKING A SUBMODULE
###############################################################################
def get_embedding(model, image_path: str, layer_index: int):
    """
    Extracts a feature embedding from YOLOv5n by hooking into a specified
    submodule index from a flattened list of submodules.
    """
    # Load image via OpenCV
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Image not found: {image_path}")

    # Convert BGR -> RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Flatten all submodules of YOLO
    all_layers = flatten_submodules(model)

    if layer_index < 0 or layer_index >= len(all_layers):
        msg = (
            f"Invalid layer_index={layer_index}. Valid range: 0..{len(all_layers)-1}\n\n"
            "Hint: Inspect the layers or reduce your chosen index."
        )
        raise IndexError(msg)

    # Hook function to capture output from the chosen layer
    features = []
    def hook_fn(module, input, output):
        features.append(output)

    chosen_layer = all_layers[layer_index]
    hook = chosen_layer.register_forward_hook(hook_fn)

    # Run inference on a single image
    _ = model([img])

    # Remove hook
    hook.remove()

    if not features:
        raise RuntimeError("Feature extraction failed: The chosen layer produced no output.")

    # Flatten e.g. [batch, channels, H, W] -> [batch, channels*H*W]
    feat = features[0].view(features[0].size(0), -1)
    # Convert to NumPy (we only had one image in the batch)
    vec = feat.detach().cpu().numpy()[0]
    return vec

###############################################################################
# 4) COSINE SIMILARITY
###############################################################################
def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray):
    """
    Compute cosine similarity between two vectors.
    Returns a float in [-1..1]. Closer to +1 => more similar.
    """
    dot = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return float(dot / (norm1 * norm2))

###############################################################################
# 5) MAIN DEMO
###############################################################################
def main():
    # Hardcoded paths to images
    image1_path = "image1.jpg"
    image2_path = "image2.jpg"
    image3_path = "image3.jpg"

    # CHOOSE A LAYER INDEX
    # Because we flatten the entire model, you can choose a wide range:
    # e.g. 0, 1, 2 ... up to len(all_layers)-1. The deeper layers tend
    # to produce more "semantic" features. Start with something like 50
    # or 100, depending on the total number of submodules.
    layer_index = 50

    print("Loading YOLOv5n model...")
    model = load_model()

    print("Extracting embeddings...")
    try:
        vec1 = get_embedding(model, image1_path, layer_index)
        vec2 = get_embedding(model, image2_path, layer_index)
        vec3 = get_embedding(model, image3_path, layer_index)
    except IndexError as e:
        print("\n[ERROR] Invalid layer index chosen.")
        print(e)
        # Optional: print submodule count or submodule names here
        # all_layers = flatten_submodules(model)
        # for i, layer in enumerate(all_layers):
        #     print(i, layer)
        return
    except Exception as e:
        print(f"[ERROR] {e}")
        return

    # Compute similarities
    sim_1_2 = cosine_similarity(vec1, vec2)
    sim_1_3 = cosine_similarity(vec1, vec3)

    print(f"\nCosine similarity between image1 and image2: {sim_1_2:.4f}")
    print(f"Cosine similarity between image1 and image3: {sim_1_3:.4f}")


if __name__ == "__main__":
    main()


Loading YOLOv5n model...


Using cache found in C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-3-1 Python-3.12.1 torch-2.5.0+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs
Adding AutoShape... 
C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Extracting embeddings...

Cosine similarity between image1 and image2: 0.8229
Cosine similarity between image1 and image3: 0.7536


C:\Users\husan/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
